#*Importing libraries*

In [1]:
import numpy as np

#*The main code of the Lifted Product code*

In [2]:
class LiftedProductCode:

    #Initialize with cyclic group order n (size of the lifting group G).
    def __init__(self, group_order):
        self.n = group_order
        self.G = self._generate_group()

    #Generate the cyclic group matrices G = [I, P, P^2, ..., P^(n-1)].
    def _generate_group(self):
        P = np.roll(np.eye(self.n), shift=1, axis=1)
        return [np.linalg.matrix_power(P, i) for i in range(self.n)]

    #Lift a base matrix by replacing entries with group matrices or zero blocks.
    def lifted_matrix(self, base_matrix):
        lifted_rows = []

        for row in base_matrix:
            block_row = []
            for val in row:
                if isinstance(val, int) and val != 0:
                    block = self.G[val % self.n]
                else:
                    block = np.zeros((self.n, self.n))
                block_row.append(block)
            lifted_rows.append(np.hstack(block_row))

        return np.vstack(lifted_rows)

    #Compute the lifted product code for matrices A and B.
    def compute_product_code(self, A, B):
        A_lift = self.lifted_matrix(A)
        B_lift = self.lifted_matrix(B)

        mA, nA = A_lift.shape
        mB, nB = B_lift.shape

        # Construct H_X
        HX_left = np.kron(A_lift, np.eye(mB))
        HX_right = np.kron(np.eye(mA), B_lift)
        HX = np.hstack((HX_left, HX_right))

        # Construct H_Z
        HZ_left = np.kron(np.eye(nA), B_lift.T)
        HZ_right = np.kron(A_lift.T, np.eye(nB))
        HZ = np.hstack((HZ_left, HZ_right))

        # Check CSS condition
        if np.all((HX @ HZ.T) % 2 == 0):
            print("CSS condition satisfied.")
            return HX, HZ
        else:
            print("CSS condition NOT satisfied.")
            return None, None


#*Accessing the .txt file for H1 matrix and getting the matrix*

In [24]:
nums_list = []

with open('/content/H1_Marix_100n_50k.txt', 'r') as file:
    for line in file:
        numbers = list(map(int, line.strip().split()))
        nums_list.append(numbers)

nums = np.array(nums_list, dtype=int)
nums = np.matrix(nums)

In [29]:
rows = 100
cols = 50
H1 = np.zeros((rows, cols))

In [32]:
num_rows, num_cols = nums.shape
for i in range(num_rows):
  for j in range(num_cols):
    if nums[i,j] != -1:
      H1[i, nums[i,j]] = 1
    else:
      continue

In [35]:
np.shape(H1)

(100, 50)

#*Preparing the H2 matrix*

*H2 is the parity check matrix of the 3-bit repetition code*

In [41]:
H2 = np.array([[1, 1, 0], [0, 1, 1]])

#*Getting the final parity check matrix of the Lifted Product code*

In [42]:
lpc = LiftedProductCode(group_order=3)

HX, HZ = lpc.compute_product_code(H1, H2)

if HX is not None:
    print("H_X shape:", HX.shape)
    print("H_Z shape:", HZ.shape)

CSS condition satisfied.
H_X shape: (1800, 3600)
H_Z shape: (1350, 3600)
